# Exercise I

### Create a function named get_db_url. It should accept a username, hostname, password, and database name and return a url connection string formatted like in the example at the start of this lesson.

In [1]:
import pandas as pd
import numpy as np
from pydataset import data

In [2]:
from env import user, password, host



In [3]:

def get_db_url(database):
    return f'mysql+pymysql://{user}:{password}@{host}/{database}'

### 
### Use your function to obtain a connection to the employees database.
### 

In [4]:
url = f'mysql+pymysql://{user}:{password}@{host}/employees'

### Once you have successfully run a query:

In [5]:
pd.read_sql('SELECT * FROM employees LIMIT 5 OFFSET 50', url)


,emp_no,birth_date,first_name,last_name,gender,hire_date
0,10051,1953-07-28,Hidefumi,Caine,M,1992-10-15
1,10052,1961-02-26,Heping,Nitsch,M,1988-05-21
2,10053,1954-09-13,Sanjiv,Zschoche,F,1986-02-04
3,10054,1957-04-04,Mayumi,Schueller,M,1995-03-13
4,10055,1956-06-06,Georgy,Dredge,M,1992-04-27


### 
### a. Intentionally make a typo in the database url. What kind of error message do you see?

In [6]:
url = f'mysql+pymysql://{user}:{password}@{host}/employes'

nothing

### 
### b. Intentionally make an error in your SQL query. What does the error message look like?

```
OperationalError: (pymysql.err.OperationalError) (1044, "Access denied for user 'quintela_2243'@'%' to database 'employes'")
(Background on this error at: https://sqlalche.me/e/14/e3q8)

```


(pymysql.err.OperationalError) (1044, "Access denied for user 'quintela_2243'@'%' to database 'employes'")
(Background on this error at: https://sqlalche.me/e/14/e3q8)


### 
### Read the employees and titles tables into two separate DataFrames.

In [7]:
url = f'mysql+pymysql://{user}:{password}@{host}/employees'

In [8]:
employees = 'SELECT * FROM employees'
titles = 'SELECT * FROM titles'

employees_df = pd.read_sql(employees, get_db_url('employees'))

employees_df.head()


,emp_no,birth_date,first_name,last_name,gender,hire_date
0,10001,1953-09-02,Georgi,Facello,M,1986-06-26
1,10002,1964-06-02,Bezalel,Simmel,F,1985-11-21
2,10003,1959-12-03,Parto,Bamford,M,1986-08-28
3,10004,1954-05-01,Chirstian,Koblick,M,1986-12-01
4,10005,1955-01-21,Kyoichi,Maliniak,M,1989-09-12


In [9]:
titles_df = pd.read_sql(titles, get_db_url('employees'))

titles_df

,emp_no,title,from_date,to_date
0,10001,Senior Engineer,1986-06-26,9999-01-01
1,10002,Staff,1996-08-03,9999-01-01
2,10003,Senior Engineer,1995-12-03,9999-01-01
3,10004,Engineer,1986-12-01,1995-12-01
4,10004,Senior Engineer,1995-12-01,9999-01-01
...,...,...,...,...
443303,499997,Engineer,1987-08-30,1992-08-29
443304,499997,Senior Engineer,1992-08-29,9999-01-01
443305,499998,Senior Staff,1998-12-27,9999-01-01
443306,499998,Staff,1993-12-27,1998-12-27


### 
### How many rows and columns do you have in each DataFrame? Is that what you expected?

In [10]:
employees_df.shape

(300024, 6)

In [11]:
titles_df.shape

(443308, 4)

### 
### Display the summary statistics for each DataFrame.

In [12]:
employees_df.describe()

,emp_no
count,300024.000000
mean,253321.763392
std,161828.235540
min,10001.000000
25%,85006.750000
50%,249987.500000
75%,424993.250000
max,499999.000000


In [13]:
titles_df.describe()

,emp_no
count,443308.000000
mean,253075.034430
std,161853.292613
min,10001.000000
25%,84855.750000
50%,249847.500000
75%,424891.250000
max,499999.000000


### 
### How many unique titles are in the titles DataFrame?

In [14]:
titles_df['title'].nunique()




7

### 
### What is the oldest date in the to_date column?

In [15]:
titles_df['to_date'].min()

datetime.date(1985, 3, 1)

### 
### What is the most recent date in the to_date column?

In [16]:
titles_df['to_date'].max()

datetime.date(9999, 1, 1)

# Exercise II

### Copy the users and roles DataFrames from the examples above.
### 

In [17]:


users = pd.DataFrame({
    'id': [1, 2, 3, 4, 5, 6],
    'name': ['bob', 'joe', 'sally', 'adam', 'jane', 'mike'],
    'role_id': [1, 2, 3, 3, np.nan, np.nan]
})
users


,id,name,role_id
0,1,bob,1.0
1,2,joe,2.0
2,3,sally,3.0
3,4,adam,3.0
4,5,jane,NaN
5,6,mike,NaN


In [18]:


roles = pd.DataFrame({
    'id': [1, 2, 3, 4],
    'name': ['admin', 'author', 'reviewer', 'commenter']
})
roles


,id,name
0,1,admin
1,2,author
2,3,reviewer
3,4,commenter


### What is the result of using a right join on the DataFrames?
### 

In [19]:


users.merge(roles, left_on='role_id', right_on='id', how='right', indicator=True)


,id_x,name_x,role_id,id_y,name_y,_merge
0,1.0,bob,1.0,1,admin,both
1,2.0,joe,2.0,2,author,both
2,3.0,sally,3.0,3,reviewer,both
3,4.0,adam,3.0,3,reviewer,both
4,NaN,NaN,NaN,4,commenter,right_only


#### duplicates of ID's need to be cleaned up for every row and NaN values for row 4 (name and Id and role id)

#### clean dup

In [20]:
(users.merge(roles, 
            left_on='role_id', 
            right_on='id', 
            how='right')
    .drop(columns='role_id')
    .rename(columns={'id_x': 'id', 
                     'name_x': 'employee',
                     'id_y': 'role_id',
                     'name_y': 'role'}
            )
)

,id,employee,role_id,role
0,1.0,bob,1,admin
1,2.0,joe,2,author
2,3.0,sally,3,reviewer
3,4.0,adam,3,reviewer
4,NaN,NaN,4,commenter


### What is the result of using an outer join on the DataFrames?
### 

In [21]:
users.merge(roles, left_on='role_id', right_on='id', how='outer', indicator=True)

,id_x,name_x,role_id,id_y,name_y,_merge
0,1.0,bob,1.0,1.0,admin,both
1,2.0,joe,2.0,2.0,author,both
2,3.0,sally,3.0,3.0,reviewer,both
3,4.0,adam,3.0,3.0,reviewer,both
4,5.0,jane,NaN,NaN,NaN,left_only
5,6.0,mike,NaN,NaN,NaN,left_only
6,NaN,NaN,NaN,4.0,commenter,right_only


### row 6 id, name, and role NaN and row 4,5 roleid, id, and name NaN

### What happens if you drop the foreign keys from the DataFrames and try to merge them?

In [22]:
 users_drop = [users.drop(columns = ['role_id', 'id'])]

In [23]:
roles_drop = [roles.drop(columns = ['id'])]

users_drop.merge(roles_drop, left_on='role_id', right_on='id', how='outer', indicator=True)

### Load the mpg dataset from PyDataset.
### 

In [25]:
mpg = data('mpg')


In [26]:
mpg

,manufacturer,model,displ,year,cyl,trans,drv,cty,hwy,fl,class
1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact
2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact
3,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact
4,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact
5,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact
...,...,...,...,...,...,...,...,...,...,...,...
230,volkswagen,passat,2.0,2008,4,auto(s6),f,19,28,p,midsize
231,volkswagen,passat,2.0,2008,4,manual(m6),f,21,29,p,midsize
232,volkswagen,passat,2.8,1999,6,auto(l5),f,16,26,p,midsize
233,volkswagen,passat,2.8,1999,6,manual(m5),f,18,26,p,midsize


### Output and read the documentation for the mpg dataset.
### 

In [27]:
data('mpg', show_doc = True)

mpg

PyDataset Documentation (adopted from R Documentation. The displayed examples are in R)

## Fuel economy data from 1999 and 2008 for 38 popular models of car

### Description

This dataset contains a subset of the fuel economy data that the EPA makes
available on http://fueleconomy.gov. It contains only models which had a new
release every year between 1999 and 2008 - this was used as a proxy for the
popularity of the car.

### Usage

    data(mpg)

### Format

A data frame with 234 rows and 11 variables

### Details

  * manufacturer. 

  * model. 

  * displ. engine displacement, in litres 

  * year. 

  * cyl. number of cylinders 

  * trans. type of transmission 

  * drv. f = front-wheel drive, r = rear wheel drive, 4 = 4wd 

  * cty. city miles per gallon 

  * hwy. highway miles per gallon 

  * fl. 

  * class. 




### How many rows and columns are in the dataset?
### 

In [28]:
mpg.shape

(234, 11)

### Check out your column names and perform any cleanup you may want on them.
### 

In [29]:
mpg_df = mpg.drop(columns = ['cyl','fl','class', 'drv'])

In [30]:
mpg_df

,manufacturer,model,displ,year,trans,cty,hwy
1,audi,a4,1.8,1999,auto(l5),18,29
2,audi,a4,1.8,1999,manual(m5),21,29
3,audi,a4,2.0,2008,manual(m6),20,31
4,audi,a4,2.0,2008,auto(av),21,30
5,audi,a4,2.8,1999,auto(l5),16,26
...,...,...,...,...,...,...,...
230,volkswagen,passat,2.0,2008,auto(s6),19,28
231,volkswagen,passat,2.0,2008,manual(m6),21,29
232,volkswagen,passat,2.8,1999,auto(l5),16,26
233,volkswagen,passat,2.8,1999,manual(m5),18,26


In [160]:
manufacturer_group = mpg_df.groupby('manufacturer').model.describe()
manufacturer_group


,count,unique,top,freq
manufacturer,,,,
audi,18,3,a4 quattro,8
chevrolet,19,4,c1500 suburban 2wd,5
dodge,37,4,caravan 2wd,11
ford,25,4,mustang,9
honda,9,1,civic,9
hyundai,14,2,sonata,7
jeep,8,1,grand cherokee 4wd,8
land rover,4,1,range rover,4
lincoln,3,1,navigator 2wd,3


### Display the summary statistics for the dataset.
### 

In [32]:
mpg_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 234 entries, 1 to 234
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   manufacturer  234 non-null    object 
 1   model         234 non-null    object 
 2   displ         234 non-null    float64
 3   year          234 non-null    int64  
 4   trans         234 non-null    object 
 5   cty           234 non-null    int64  
 6   hwy           234 non-null    int64  
dtypes: float64(1), int64(3), object(3)
memory usage: 14.6+ KB


### How many different manufacturers are there?
### 

In [161]:
manufacturer_group.count()

count     15
unique    15
top       15
freq      15
dtype: int64

### How many different models are there?
### 

In [34]:
model_group = mpg_df.groupby('model').year.describe()
model_group


,count,mean,std,min,25%,50%,75%,max
model,,,,,,,,
4runner 4wd,6.0,2002.000000,4.647580,1999.0,1999.00,1999.0,2005.75,2008.0
a4,7.0,2002.857143,4.810702,1999.0,1999.00,1999.0,2008.00,2008.0
a4 quattro,8.0,2003.500000,4.810702,1999.0,1999.00,2003.5,2008.00,2008.0
a6 quattro,3.0,2005.000000,5.196152,1999.0,2003.50,2008.0,2008.00,2008.0
altima,6.0,2005.000000,4.647580,1999.0,2001.25,2008.0,2008.00,2008.0
c1500 suburban 2wd,5.0,2006.200000,4.024922,1999.0,2008.00,2008.0,2008.00,2008.0
camry,7.0,2002.857143,4.810702,1999.0,1999.00,1999.0,2008.00,2008.0
camry solara,7.0,2002.857143,4.810702,1999.0,1999.00,1999.0,2008.00,2008.0
caravan 2wd,11.0,2003.090909,4.700097,1999.0,1999.00,1999.0,2008.00,2008.0


In [35]:
model_group.count()

count    38
mean     38
std      38
min      38
25%      38
50%      38
75%      38
max      38
dtype: int64

### Create a column named mileage_difference like you did in the DataFrames exercises; this column should contain the difference between highway and city mileage for each car.
### 

In [36]:
mile_diff = mpg.assign(mileage_difference= mpg['hwy'] - mpg['cty'])

mile_diff

,manufacturer,model,displ,year,cyl,trans,drv,cty,hwy,fl,class,mileage_difference
1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact,11
2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact,8
3,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact,11
4,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact,9
5,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact,10
...,...,...,...,...,...,...,...,...,...,...,...,...
230,volkswagen,passat,2.0,2008,4,auto(s6),f,19,28,p,midsize,9
231,volkswagen,passat,2.0,2008,4,manual(m6),f,21,29,p,midsize,8
232,volkswagen,passat,2.8,1999,6,auto(l5),f,16,26,p,midsize,10
233,volkswagen,passat,2.8,1999,6,manual(m5),f,18,26,p,midsize,8


### Create a column named average_mileage like you did in the DataFrames exercises; this is the mean of the city and highway mileage.
### 

In [37]:
mile_diff['average_mileage'] = mile_diff[['hwy', 'cty']].mean(axis=1)
mile_diff

,manufacturer,model,displ,year,cyl,trans,drv,cty,hwy,fl,class,mileage_difference,average_mileage
1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact,11,23.5
2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact,8,25.0
3,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact,11,25.5
4,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact,9,25.5
5,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact,10,21.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
230,volkswagen,passat,2.0,2008,4,auto(s6),f,19,28,p,midsize,9,23.5
231,volkswagen,passat,2.0,2008,4,manual(m6),f,21,29,p,midsize,8,25.0
232,volkswagen,passat,2.8,1999,6,auto(l5),f,16,26,p,midsize,10,21.0
233,volkswagen,passat,2.8,1999,6,manual(m5),f,18,26,p,midsize,8,22.0


### Create a new column on the mpg dataset named is_automatic that holds boolean values denoting whether the car has an automatic transmission.
### 

In [38]:
mile_diff['is_automatic'] = mile_diff['trans'].str.contains('auto')
mile_diff

,manufacturer,model,displ,year,cyl,trans,drv,cty,hwy,fl,class,mileage_difference,average_mileage,is_automatic
1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact,11,23.5,True
2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact,8,25.0,False
3,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact,11,25.5,False
4,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact,9,25.5,True
5,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact,10,21.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
230,volkswagen,passat,2.0,2008,4,auto(s6),f,19,28,p,midsize,9,23.5,True
231,volkswagen,passat,2.0,2008,4,manual(m6),f,21,29,p,midsize,8,25.0,False
232,volkswagen,passat,2.8,1999,6,auto(l5),f,16,26,p,midsize,10,21.0,True
233,volkswagen,passat,2.8,1999,6,manual(m5),f,18,26,p,midsize,8,22.0,False


### Using the mpg dataset, find out which which manufacturer has the best miles per gallon on average?
### 

In [39]:
best_mileage = mile_diff.groupby('manufacturer').average_mileage.max()
highest_mileage = best_mileage.idxmax()
highest_mileage



'volkswagen'

### Do automatic or manual cars have better miles per gallon?
### 
### 

#### Highest specific transmission

In [40]:
best_mileage = mile_diff.groupby('trans').average_mileage.max()
highest_mileage = best_mileage.idxmax()
highest_mileage

'manual(m5)'

#### manuals vs automatic: Average City Mileage

In [41]:
auto_avg_mpg = mpg[mpg['trans'].str.contains('auto')]['cty'].mean()

man_avg_mpg = mpg[mpg['trans'].str.contains('man')]['cty'].mean()

if auto_avg_mpg > man_avg_mpg:
    print('Automatics are more fuel effecient in the city')
else:
    print('Manuals are more fuel effecient in the city')



Manuals are more fuel effecient in the city


#### manuals vs automatic: Average Highway Mileage

In [42]:
auto_avg_mpg = mpg[mpg['trans'].str.contains('auto')]['hwy'].mean()

man_avg_mpg = mpg[mpg['trans'].str.contains('man')]['hwy'].mean()

if auto_avg_mpg > man_avg_mpg:
    print('Automatics are more fuel effecient on the highway')
else:
    print('Manuals are more fuel effecient on the highway')

Manuals are more fuel effecient on the highway


# Exercise III

### Use your get_db_url function to help you explore the data from the chipotle database.
### 

In [53]:
from env import user, password, host
def get_db_url(database):
    return f'mysql+pymysql://{user}:{password}@{host}/{database}'

In [54]:
url = f'mysql+pymysql://{user}:{password}@{host}/chipotle'

### What is the total price for each order?
### 

In [80]:
chip_df = pd.read_sql('SELECT * FROM orders', url)
chip_df['item_price']= chip_df['item_price'].str.replace('$','').astype(float)





/var/folders/8g/xqsgr3tj7qn4ffm_2xh9lqym0000gn/T/ipykernel_54918/3331092624.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  chip_df['item_price']= chip_df['item_price'].str.replace('$','').astype(float)


In [81]:
chip_df.dtypes

id                      int64
order_id                int64
quantity                int64
item_name              object
choice_description     object
item_price            float64
dtype: object

In [82]:
chip_df = chip_df.assign(total_price = chip_df['quantity'] * chip_df['item_price'])
chip_df

,id,order_id,quantity,item_name,choice_description,item_price,total_price
0,1,1,1,Chips and Fresh Tomato Salsa,nan,2.39,2.39
1,2,1,1,Izze,[Clementine],3.39,3.39
2,3,1,1,Nantucket Nectar,[Apple],3.39,3.39
3,4,1,1,Chips and Tomatillo-Green Chili Salsa,nan,2.39,2.39
4,5,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans, Rice, Cheese, Sour Cream]]",16.98,33.96
...,...,...,...,...,...,...,...
4617,4618,1833,1,Steak Burrito,"[Fresh Tomato Salsa, [Rice, Black Beans, Sour Cream, Cheese, Lettuce, Guacamole]]",11.75,11.75
4618,4619,1833,1,Steak Burrito,"[Fresh Tomato Salsa, [Rice, Sour Cream, Cheese, Lettuce, Guacamole]]",11.75,11.75
4619,4620,1834,1,Chicken Salad Bowl,"[Fresh Tomato Salsa, [Fajita Vegetables, Pinto Beans, Guacamole, Lettuce]]",11.25,11.25
4620,4621,1834,1,Chicken Salad Bowl,"[Fresh Tomato Salsa, [Fajita Vegetables, Lettuce]]",8.75,8.75


In [90]:
total_sum = chip_df.groupby('order_id').total_price.agg('sum')

total_sum.reset_index()


,order_id,total_price
0,1,11.56
1,2,33.96
2,3,12.67
3,4,21.00
4,5,13.70
...,...,...
1829,1830,23.00
1830,1831,12.90
1831,1832,13.20
1832,1833,23.50


### What are the most popular 3 items?
### 

In [97]:
most_pop = chip_df.groupby('item_name').quantity.agg('sum').sort_values()
most_pop.tail(3)

item_name
Chips and Guacamole    506
Chicken Burrito        591
Chicken Bowl           761
Name: quantity, dtype: int64

### Which item has produced the most revenue?
### 

In [118]:
chip_df.groupby('item_name').total_price.agg('sum').sort_values().tail(1)



item_name
Chicken Bowl    8044.63
Name: total_price, dtype: float64

### Join the employees and titles DataFrames together.
### 

In [119]:
url = f'mysql+pymysql://{user}:{password}@{host}/employees'

In [121]:
employees = 'SELECT * FROM employees'
titles = 'SELECT * FROM titles'

employees_df = pd.read_sql(employees, get_db_url('employees'))

employees_df.head()

,emp_no,birth_date,first_name,last_name,gender,hire_date
0,10001,1953-09-02,Georgi,Facello,M,1986-06-26
1,10002,1964-06-02,Bezalel,Simmel,F,1985-11-21
2,10003,1959-12-03,Parto,Bamford,M,1986-08-28
3,10004,1954-05-01,Chirstian,Koblick,M,1986-12-01
4,10005,1955-01-21,Kyoichi,Maliniak,M,1989-09-12


In [124]:
titles_df = pd.read_sql(titles, get_db_url('employees'))

titles_df

,emp_no,title,from_date,to_date
0,10001,Senior Engineer,1986-06-26,9999-01-01
1,10002,Staff,1996-08-03,9999-01-01
2,10003,Senior Engineer,1995-12-03,9999-01-01
3,10004,Engineer,1986-12-01,1995-12-01
4,10004,Senior Engineer,1995-12-01,9999-01-01
...,...,...,...,...
443303,499997,Engineer,1987-08-30,1992-08-29
443304,499997,Senior Engineer,1992-08-29,9999-01-01
443305,499998,Senior Staff,1998-12-27,9999-01-01
443306,499998,Staff,1993-12-27,1998-12-27


In [157]:
emp_title = employees_df.merge(titles_df, left_on='emp_no', right_on='emp_no', how='right')
emp_title

,emp_no,birth_date,first_name,last_name,gender,hire_date,title,from_date,to_date
0,10001,1953-09-02,Georgi,Facello,M,1986-06-26,Senior Engineer,1986-06-26,9999-01-01
1,10002,1964-06-02,Bezalel,Simmel,F,1985-11-21,Staff,1996-08-03,9999-01-01
2,10003,1959-12-03,Parto,Bamford,M,1986-08-28,Senior Engineer,1995-12-03,9999-01-01
3,10004,1954-05-01,Chirstian,Koblick,M,1986-12-01,Engineer,1986-12-01,1995-12-01
4,10004,1954-05-01,Chirstian,Koblick,M,1986-12-01,Senior Engineer,1995-12-01,9999-01-01
...,...,...,...,...,...,...,...,...,...
443303,499997,1961-08-03,Berhard,Lenart,M,1986-04-21,Engineer,1987-08-30,1992-08-29
443304,499997,1961-08-03,Berhard,Lenart,M,1986-04-21,Senior Engineer,1992-08-29,9999-01-01
443305,499998,1956-09-05,Patricia,Breugel,M,1993-10-13,Senior Staff,1998-12-27,9999-01-01
443306,499998,1956-09-05,Patricia,Breugel,M,1993-10-13,Staff,1993-12-27,1998-12-27


### For each title, find the hire date of the employee that was hired most recently with that title.

In [158]:
result = emp_title.groupby('title')['hire_date'].max()
result


title
Assistant Engineer    1999-12-24
Engineer              2000-01-28
Manager               1992-02-05
Senior Engineer       2000-01-01
Senior Staff          2000-01-13
Staff                 2000-01-12
Technique Leader      1999-12-31
Name: hire_date, dtype: object

### 
### Write the code necessary to create a cross tabulation of the number of titles by department. (Hint: this will involve a combination of SQL code to pull the necessary data and python/pandas code to perform the manipulations.)

In [162]:
dept_title = '''
                    SELECT t.emp_no,
                    t.title, 
                    t.from_date, 
                    t.to_date, 
                    d.dept_name 
                    FROM departments AS d 
                    JOIN dept_emp AS de USING(dept_no) 
                    JOIN titles AS t USING(emp_no);
                    '''

In [163]:

dept_titles = pd.read_sql(dept_title, get_db_url('employees'))


dept_titles.head()

,emp_no,title,from_date,to_date,dept_name
0,10011,Staff,1990-01-22,1996-11-09,Customer Service
1,10038,Senior Staff,1996-09-20,9999-01-01,Customer Service
2,10038,Staff,1989-09-20,1996-09-20,Customer Service
3,10049,Senior Staff,2000-05-04,9999-01-01,Customer Service
4,10049,Staff,1992-05-04,2000-05-04,Customer Service


In [164]:
dept_titles.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 489903 entries, 0 to 489902
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   emp_no     489903 non-null  int64 
 1   title      489903 non-null  object
 2   from_date  489903 non-null  object
 3   to_date    489903 non-null  object
 4   dept_name  489903 non-null  object
dtypes: int64(1), object(4)
memory usage: 18.7+ MB


In [165]:
all_titles_crosstab = pd.crosstab(
    dept_titles.dept_name, dept_titles.title)


In [166]:
all_titles_crosstab


title,Assistant Engineer,Engineer,Manager,Senior Engineer,Senior Staff,Staff,Technique Leader
dept_name,,,,,,,
Customer Service,298,2362,4,2027,13925,16150,309
Development,7769,58135,2,49326,1247,1424,7683
Finance,0,0,2,0,12139,13929,0
Human Resources,0,0,2,0,12274,14342,0
Marketing,0,0,2,0,13940,16196,0
Production,6445,49649,4,42205,1270,1478,6557
Quality Management,1831,13852,4,11864,0,0,1795
Research,378,2986,2,2570,11637,13495,393
Sales,0,0,2,0,36191,41808,0


In [167]:
current_titles = dept_titles[dept_titles.to_date == dept_titles.to_date.max()]
current_titles.head()

,emp_no,title,from_date,to_date,dept_name
1,10038,Senior Staff,1996-09-20,9999-01-01,Customer Service
3,10049,Senior Staff,2000-05-04,9999-01-01,Customer Service
5,10060,Senior Staff,1996-05-28,9999-01-01,Customer Service
7,10088,Senior Staff,1993-09-02,9999-01-01,Customer Service
11,10112,Staff,1998-05-01,9999-01-01,Customer Service


In [168]:
current_titles_crosstab = pd.crosstab(current_titles.dept_name, current_titles.title)
current_titles_crosstab

title,Assistant Engineer,Engineer,Manager,Senior Engineer,Senior Staff,Staff,Technique Leader
dept_name,,,,,,,
Customer Service,68,627,1,1790,12349,3902,241
Development,1833,15677,1,43364,1085,315,6117
Finance,0,0,1,0,10650,3199,0
Human Resources,0,0,1,0,10843,3416,0
Marketing,0,0,1,0,12371,3880,0
Production,1542,13325,1,37156,1123,349,5210
Quality Management,433,3744,1,10390,0,0,1422
Research,77,830,1,2250,10219,3206,321
Sales,0,0,1,0,32032,9967,0
